perform sentiment analysis (Using Naive Bayes Classifier)

In [1]:
# importing the dependencies
import numpy as np
import pickle
import collections
import nltk.classify.util, nltk.metrics
from nltk.classify import NaiveBayesClassifier, MaxentClassifier, SklearnClassifier
import csv
from sklearn import cross_validation
from sklearn.svm import LinearSVC, SVC
import random
from nltk.corpus import stopwords
import itertools
from nltk.collocations import BigramCollocationFinder
from nltk.metrics import BigramAssocMeasures
import pandas as pd

C:\Users\ADITYA\Anaconda3\envs\ml\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


using unigrams features

In [2]:
#Finding the unigram representation
from sklearn.feature_extraction.text import CountVectorizer
vectorizer=CountVectorizer()

In [3]:
# loading the cleaned training tweets
with open('sentiments/SentimentText_train.txt', 'rb') as f:
    SentimentText = pickle.load(f)

In [4]:
# fitting the vectorizer
X=vectorizer.fit_transform(SentimentText)

In [5]:
# getting the target values i.e wheather the tweets are positive or negative
train_data = pd.read_csv('data/train.csv', encoding = 'ISO-8859-1')
y = train_data['Sentiment']

In [6]:
# checking the shape of the output variable
y.shape

(99989,)

In [7]:
# checking the shape of the input matrix -- this is feed into the classifier for training
X.shape

(99989, 41679)

In [8]:
# splitting into training and testing set so as to evaluate our classifier
from sklearn.model_selection import train_test_split
X_train, X_test, y_train ,y_test = train_test_split(X,y,train_size = .8 , test_size = .2 , random_state = 0)

In [9]:
# fitting the multinomial bayes classifier
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(X, y)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [10]:
# training score
clf.score(X_train,y_train)

0.8181795451988348

In [11]:
# testing score
clf.score(X_test, y_test)

0.8166316631663166

In [ ]:
# finally loading the cleaned test tweets 
with open('sentiments/SentimentText_test.txt', 'rb') as f:
    SentimentText = pickle.load(f)

In [ ]:
X_test=vectorizer.transform(SentimentText)

In [ ]:
X_test.shape

In [ ]:
predicted = clf.predict(X_test)

In [ ]:
predicted.shape

In [ ]:
# this code generates output csv on the test set which is submitted to the kaggle for score evaluation

output = pd.read_csv('data/test.csv', encoding = 'ISO-8859-1')
output.drop(['SentimentText'],axis = 1)
output['Sentiment'] = predicted
output.to_csv('output.csv', index = False)

using bigrams features

In [12]:
# #Finding the bigram representation 
bigram_vectorizer=CountVectorizer(ngram_range=(1,2))

In [13]:
with open('sentiments/SentimentText_train.txt', 'rb') as f:
    SentimentText = pickle.load(f)

In [14]:
X=bigram_vectorizer.fit_transform(SentimentText)

In [15]:
train_data = pd.read_csv('data/train.csv', encoding = 'ISO-8859-1')
y = train_data['Sentiment']

In [16]:
X.shape

(99989, 428766)

In [17]:
y.shape

(99989,)

In [18]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train ,y_test = train_test_split(X,y,train_size = .8 , test_size = .2 , random_state = 0)

In [19]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(X, y)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [20]:
# training score
clf.score(X_train, y_train)

0.9352552162118238

In [21]:
# testing score
clf.score(X_test, y_test)

0.9336433643364337

In [22]:
with open('sentiments/SentimentText_test.txt', 'rb') as f:
    SentimentText = pickle.load(f)

In [26]:
X_test=bigram_vectorizer.transform(SentimentText)

In [27]:
X_test.shape

(299989, 428766)

In [28]:
predicted = clf.predict(X_test)

In [29]:
predicted.shape

(299989,)

In [30]:
# this code generates output csv on the test set which is submitted to the kaggle for score evaluation

output = pd.read_csv('data/test.csv', encoding = 'ISO-8859-1')
output.drop(['SentimentText'],axis = 1)
output['Sentiment'] = predicted
output.to_csv('output.csv', index = False)

using both unigrams and bigrams as features set

In [31]:
#Finding the unigram representation
from sklearn.feature_extraction.text import CountVectorizer
vectorizer=CountVectorizer()

# #Finding the bigram representation 
bigram_vectorizer=CountVectorizer(ngram_range=(1,2))

In [32]:
with open('sentiments/SentimentText_train.txt', 'rb') as f:
    SentimentText = pickle.load(f)

In [33]:
X_unigrams = vectorizer.fit_transform(SentimentText)
X_bigrams = bigram_vectorizer.fit_transform(SentimentText)


In [34]:
X_unigrams.shape

(99989, 41679)

In [35]:
X_bigrams.shape

(99989, 428766)

In [36]:
from scipy.sparse import hstack 
X_combined = hstack([X_unigrams , X_bigrams])

In [37]:
X_combined.shape

(99989, 470445)

In [38]:
train_data = pd.read_csv('data/train.csv', encoding = 'ISO-8859-1')
y = train_data['Sentiment']

In [39]:
clf = MultinomialNB()
clf.fit(X_combined, y)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [40]:
with open('sentiments/SentimentText_test.txt', 'rb') as f:
    SentimentText = pickle.load(f)

In [41]:
X_test_unigrams = vectorizer.transform(SentimentText)
X_test_bigrams = bigram_vectorizer.transform(SentimentText)

In [42]:
X_test_unigrams.shape

(299989, 41679)

In [43]:
X_test_bigrams.shape

(299989, 428766)

In [44]:
from scipy.sparse import hstack 
X_test_combined = hstack([X_test_unigrams , X_test_bigrams])

In [45]:
X_test_combined.shape

(299989, 470445)

In [46]:
predicted = clf.predict(X_test_combined)

In [47]:
# this code generates output csv on the test set which is submitted to the kaggle for score evaluation

output = pd.read_csv('data/test.csv', encoding = 'ISO-8859-1')
output.drop(['SentimentText'],axis = 1)
output['Sentiment'] = predicted
output.to_csv('output.csv', index = False)